In [ ]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Importation de données

In [ ]:
df = pd.read_excel('../dataset_indice_prix_moyens.xlsx', header=0)

In [ ]:
df.head()

# Séléction des variables

In [ ]:
variables = list(df.columns[:-1]) #ici nous recuperons tous les variables sauf la variable explicative

variables

# Correlation entre les variables

In [ ]:
df[variables].corr(method='spearman')

# Codification des variables qualitatives 

In [ ]:
convertion_dict = {'MOIS': {'JANVIER': 1, 'FEVRIER' : 2, 'MARS': 3, 'AVRIL' : 4, 'MAI' : 5, 'JUIN' : 6, 'JUILLET' : 7, 'AOUT' : 8, 'SEPTEMBRE' : 9, 'OCTOBRE' : 10, 'NOVEMBRE' : 11, 'DECEMBRE' : 12}}

df.replace(to_replace=convertion_dict, inplace=True)

df.head()

In [ ]:
df[variables].corr(method='spearman').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'))

In [ ]:
variables_new = list(df.columns[2:-1]) #ici nous recuperons toutes les variables qui sont correlées

variables_new

# chargement Matrice X

In [ ]:
X = df[variables_new]

X.head()

# Analyse descriptive

In [ ]:
X.describe()

# chargement de vecteur classe y

In [ ]:
y = df[df.columns[-1]]

y.head()

In [ ]:
plt.hist(y)

# Normalisation et gestion de valeurs manquantes

In [ ]:
imputer_model = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer_model.fit(X)

X = imputer_model.transform(X)

X

In [ ]:
for x in X:
    plt.hist(x)

### Segmentation de l'ensemble de données en sous-ensembles d'entraînement (training set) et de test (test set)
Nous optons de faire une segmentation de l'ensemble de données en ensembles d'apprentissage et de test comprenant chacun repectivement <b>70%</b> et <b>30%</b> des données.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state = 0)

### Création du modèle

a. Regression Linéaire

In [ ]:
modele_regression = linear_model.LinearRegression()

#### Apprentissage

In [ ]:
modele_regression.fit(X_train, y_train)

In [ ]:
precision_reg = modele_regression.score(X_test, y_test)

test_error_regression = np.mean((modele_regression.predict(X_test) - y_test) ** 2)

train_error_regression = np.mean((modele_regression.predict(X_train) - y_train) ** 2)

print('Coefficients: \n', modele_regression.coef_) # coefficients

print ('intercept', modele_regression.intercept_) # la constante

print("Residual sum of squares test: %.2f" % test_error_regression) #Erreur quadratique moyenne

print("Residual sum of squares train: %.2f" % train_error_regression) #Erreur quadratique moyenne

print("La précision du modèle de regression est de", np.round(precision_reg * 100, 2), "%")

#### fonction de prédiction de modèle de regression multiple

indice = ∑〖βi*Xi〗+ b

In [ ]:
i = 0
model = ''
for x in modele_regression.coef_:
        model = model + str(x)+' * X'+str(i + 1) + ' + '
        i = i + 1
    
model = 'y_predited(INDICE) = '+ model + str(modele_regression.intercept_)

model

b. Forêt aléatoire modele de regression

In [ ]:
modele_forest = RandomForestRegressor()

#### Apprentissage

In [ ]:
modele_forest.fit(X_train, y_train)

In [ ]:
y_forest_predited = modele_forest.predict(X_test)

score = r2_score(y_test, y_forest_predited)

test_error_forest = np.mean((y_forest_predited - y_test) ** 2)

train_error_forest = np.mean((modele_forest.predict(X_train) - y_train) ** 2)

print ('nombre d\'arbre est', len(modele_forest.estimators_)) # le nombre d'arbre

print("La précision du modèle de forêt aléatoire est de", np.round(score * 100, 2), "%")

print("Residual sum of squares test: %.2f" % test_error_forest) #Erreur quadratique moyenne

print("Residual sum of squares train: %.2f" % train_error_forest) #Erreur quadratique moyenne

c. Arbre de decison modèle de regression

In [ ]:
modele_arb = tree.DecisionTreeRegressor()

#### Apprentissage

In [ ]:
modele_arb.fit(X_train, y_train)

In [ ]:
precision_arb = modele_arb.score(X_test, y_test)

test_error_arb = np.mean((modele_arb.predict(X_test) - y_test) ** 2)

train_error_arb = np.mean((modele_arb.predict(X_train) - y_train) ** 2)

print("Residual sum of squares test: %.2f" % test_error_arb) #Erreur quadratique moyenne

print("Residual sum of squares train: %.2f" % train_error_arb) #Erreur quadratique moyenne

print("La précision du modèle est de", np.round(precision_arb * 100, 2), "%")

## Comparaison des modèles

Regression multiple, Forêt aléatoire de regression et Arbre de decision de regression

In [ ]:

print("La précision du modèle :\n Regression multiple est de ",np.round(precision_reg * 100, 2), "%", 
      "\n Forêt aléatoire Regression est de ", np.round(score * 100, 2), "%",
     "\n Arbre de decision Regression est de", np.round(precision_arb * 100, 2), "%"
     )

In [ ]:
train_error = [train_error_regression, train_error_forest, train_error_arb]

test_error = [test_error_regression, test_error_forest, test_error_arb]

precision_model = [np.round(precision_reg * 100, 2), np.round(score * 100, 2), np.round(precision_arb * 100, 2)]

data = {'Train Error' : train_error, 'Test Error' : test_error, 'Précision modele %' : precision_model}

models = ['Regression Multiple', 'Forêt alétaoire Regression', 'Arbre de decison regression']

df_compare = pd.DataFrame(data, index = models)

df_compare

Après analyse, nous avons constacté ce qui suit :
    - le modèle de regression multiple prédit à 100% et elle commet moins d'erreurs lors de l'apprentissage et teste
    - le modèle de Forêt aléatoire prédit à 94,75%
    - le modèle d'Arbre de décision prédit à 96,74%

In [ ]:
df_compare.plot(kind='bar')

### Prédiction de nouvelle données

chargement de nouvelles données

In [ ]:
INPUT = [[2012, 'JANVIER', 4.83, 4.18, 4.91, 3.44, 4.93, 6.14, 2.24, 2.06, 2.56, 1.11, 4.60, 2.97],
        [2022, 'FEVRIER', 0.45, 0.12, 0.14, 0.58, 0.25, 0.21, 0.33, 1.32, 0.25, 0.09, 0.29, 0.29]]

INPUT

df_new = pd.DataFrame(data = INPUT, columns=variables)

#Normalisation des données
new_X = df_new[variables_new]

new_X = imputer_model.transform(new_X)

new_X

Prédiction avec le modèle de Regression

In [ ]:
y_predited_reg = modele_regression.predict(new_X)
y_predited_reg

Prédiction avec le modèle de forêt aléatoire

In [ ]:
y_predited_forest = modele_forest.predict(new_X)
y_predited_forest

Prédiction avec le modèle d'abre de decison rgression

In [ ]:
y_predited_arb = modele_arb.predict(new_X)
y_predited_arb

In [ ]:
dict ={'INDICE GENERAL REGRESSION' : y_predited_reg, 'INDICE GENERAL FOREST' : y_predited_forest, 'INDICE GENERAL ARBRE' : y_predited_arb}

df_clf = pd.DataFrame(dict, columns =['INDICE GENERAL REGRESSION', 'INDICE GENERAL FOREST', 'INDICE GENERAL ARBRE'])

out = pd.merge(df_new, df_clf , right_index=True, left_index=True)

out

# Variation des indices

(IPCy-1 - IPCy+1)/IPCy-1  x 0.01

In [ ]:
convertion_dict = {'MOIS': {1 : 'JANVIER', 2 : 'FEVRIER', 3: 'MARS', 4 : 'AVRIL', 5 : 'MAI', 6 : 'JUIN', 7 : 'JUILLET', 8 : 'AOUT', 9 : 'SEPTEMBRE', 10 :'OCTOBRE', 11 : 'NOVEMBRE', 12 : 'DECEMBRE'}}

def get_indice_in_df(df, year, month) : #cette méthode nous permet de récupérer l'indice dans le df en fonction de l'année et mois
    indice = 0.0 
    df.replace(to_replace=convertion_dict, inplace=True)
    df = df.reset_index()  # make sure indexes pair with number of rows

    for index, row in df.iterrows():
        if row['MOIS'] == (month) and row['ANNEE'] == year :
            indice = row['INDICE GENERAL']
            break
            
    return indice
    
min_v = df['ANNEE'].min() #ici nous recuperons l'année min
max_v = df['ANNEE'].max() #ici nous récuperons l'année max

for m in ['JANVIER', 'FEVRIER', 'MARS', 'AVRIL', 'MAI', 'JUIN', 'JUILLET', 'AOUT', 'SEPTEMBRE', 'OCTOBRE', 'NOVEMBRE', 'DECEMBRE']:
    for y in range(min_v, max_v) :
        old = get_indice_in_df(df, y, m)
        new = get_indice_in_df(df, y + 1, m)
        
        old = 0 if old is None else old
        new = 0 if new is None else new
        
        variation = 0 if old is None or new is None else ((old - new)/old) * 0.01     
        
        print("I(", m, ") = ((", old, "(", y, ") - ", new, "(", y + 1, ")) / ", old, "(", y, ")) * 0.01 = ", variation, "\n")